<div align = 'center'>
    <h1 id="section-0"><font face = 黑体> 税务风险指标预警值计算 </font>
<br>
<div>

<a id="ref11"></a>
### 实验步骤要求
<div class="alert alert-block alert-warning" style="margin-top: 20px">
编写函数完成以下两个目标：
<li>1. 依据上市公司财报数据和年报数据计算企业税务风险指标</li>
<li>2. 从企业税务风险指标中推算出行业税务风险指标预警值</li>
</div>

<a id = "ref12"></a>
### 数据说明
<div class="alert alert-block alert-success" style="margin-top: 20px">
行业税务风险指标预警值计算涉及数据包括如下两部分内容：<br>
<li>1. 上市公司财报数据：包括所选行业中部分企业的利润表、现金流量表以及资产负债表（.xlsx格式），存放路径为 "data/电商行业财报数据"</li>
<li>2. 上市公司年报数据：所选行业中部分企业的年度报告（.pdf格式），您可以在进入实验环境前点击左下角 "行业年报数据下载" 按钮将数据下载到本地查看</li>

您可以从上市公司财报中提取数据用于大部分所选指标和自设指标的计算，关于财报数据的处理，平台已经内置代码如下。

“单位产成品原材料耗用率”、“固定资产综合折旧率”、“进项税额变动率与销项税额变动率配比”、“投入产出评估分析指标”以及您自设的部分指标计算所涉及数据无法从上市公司财报中获取，<strong>您需要从年报数据中提取出有效信息传入实验平台</strong>，具体操作流程如下：

<li>1. 下载 data 文件夹下的“年报信息提取模板.xlsx”文件以及 10-20 个企业的年报数据</li>
<li>2. 详细阅读年报数据，从中抽取出您认为在计算过程中需要用到的信息并按照模板文件中的格式填写（各项指标的计算公式已在文件 “data/指标计算公式.txt” 中给出）</li>
<li>3. 点击实验环境数据区上方的“上传文件”图标将编辑后的“年报信息提取模板.xlsx”文件传入数据区，并移动到 data 文件夹下覆盖原文件</li>  

说明：“单位产成品原材料耗用率”、“固定资产综合折旧率”、“进项税额变动率与销项税额变动率配比”、“投入产出评估分析指标”计算所涉及数据可能也无法从年报中提取完全，如果您选择了这些指标，<strong>在本步骤可以不用对其进行任何操作，包括提取数据</strong>。

“印花税税负变动系数”、“印花税同步增长系数”、“资源税税负变动系数”和“资源税同步增长系数”四个指标的计算数据无法从上市公司年报获取，因此<strong>这些指标不参与预警值的计算</strong>，如果您选择了其中的一项或多项指标，<strong>在本步骤无需对其进行任何操作</strong>。

注：财报数据与年报数据中涉及的企业不完全重复
</div>

<a id = "ref13"></a>
### 财报数据导入

In [1]:
import pandas as pd

# pd.set_option('future.no_silent_downcasting', True)

df1 = pd.read_excel("data/电商行业财报数据/利润表.xlsx", skiprows = 1)
df1 = df1.iloc[1:,:]

df2 = pd.read_excel("data/电商行业财报数据/现金流量表.xlsx", skiprows = 1)
df2 = df2.iloc[1:,:]

df3 = pd.read_excel("data/电商行业财报数据/资产负债表.xlsx", skiprows = 1)
df3 = df3.iloc[1:,:]

df1['证券代码'] = df1['证券代码'].astype(int)
df1['证券代码'] = df1['证券代码'].apply(lambda x: f'{x:06d}')

df2['证券代码'] = df2['证券代码'].astype(int)
df2['证券代码'] = df2['证券代码'].apply(lambda x: f'{x:06d}')

df3['证券代码'] = df3['证券代码'].astype(int)
df3['证券代码'] = df3['证券代码'].apply(lambda x: f'{x:06d}')

df12 = pd.merge(df1, df2, on = ['证券代码', '证券简称', '统计截止日期', '报表类型'], how = 'inner')
df = pd.merge(df12, df3, on = ['证券代码', '证券简称', '统计截止日期', '报表类型'], how = 'inner')

print(df.shape)

# df.head()

(57, 278)


<a id = "ref14"></a>
### 年报数据导入（需要导入年报数据时取消下一代码块中代码注释：选中代码按 Ctrl + / 取消注释）

In [2]:
# df4 = pd.read_excel("data/年报信息提取模板.xlsx", skiprows = 1)

# df4['证券代码'] = df4['证券代码'].astype(int)
# df4['证券代码'] = df4['证券代码'].apply(lambda x: f'{x:06d}')

# df = pd.merge(df, df4, on = ['证券代码', '证券简称', '统计截止日期'], how = 'left')

# print(df.shape)

# # df.head()

<a id = "ref15"></a>
### 数据预处理

In [3]:
df.iloc[:,4:] = df.iloc[:,4:].astype(float)

df = df.fillna(0.00)

## df.describe(include='all').T

## 不同证券简称可能对应不同证券代码，如'*ST 科华'和'科华生物'的证券代码都是002022。
# print(df[~df.duplicated(subset = '证券代码', keep = False)].reset_index(drop = True))

# df = df[df.duplicated(subset = '证券代码', keep = False)].reset_index(drop = True)

# df = df[~df.duplicated(subset = '证券代码', keep = False)].reset_index(drop = True)

## 去除数据量不足的证券
value_counts = df['证券代码'].value_counts()
filter_values = value_counts[value_counts > 2].index
df = df[df['证券代码'].isin(filter_values)]

print(df.shape)

df_2021 = df[df['统计截止日期'] == '2021-12-31'].reset_index(drop = True)  ## 前期数据
df_2022 = df[df['统计截止日期'] == '2022-12-31'].reset_index(drop = True)  ## 基期数据
df_2023 = df[df['统计截止日期'] == '2023-12-31'].reset_index(drop = True)  ## 本期数据

print(df_2021.shape)
print(df_2022.shape)
print(df_2023.shape)

# df.head() ## 运行此行代码，查看数据

(54, 278)
(18, 278)
(18, 278)
(18, 278)


C:\Users\16894\AppData\Local\Temp\ipykernel_52864\2425677481.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(0.00)


<a id = "ref16"></a>
### 步骤 1：计算企业税务风险指标

<a id = "ref17"></a>
### 企业指标计算说明
<div class="alert alert-block alert-info" style="margin-top: 20px">
&emsp;&emsp;企业税务风险指标的计算依托于"利润表、现金流量表以及资产负债表"以及"年度报告"（如有）中的数据，该部分数据已被读取并拆分到 df_2021、df_2022 和 df_2023 中，其中 df_2021 表示企业前期数据（统计范围为 2021-01-01 到 2021-12-31），df_2022 表示企业基期数据（统计范围为 2022-01-01 到 2022-12-31），df_2023 表示企业本期数据（统计范围为 2023-01-01 到 2023-12-31）。df_2021、df_2022 和 df_2023 三表中所统计的企业相同。<br>
&emsp;&emsp;关于具体代码的编写，下方预定义了一个名为 FinancialIndicators 的类（Class），其中 df_2021、df_2022 和 df_2023 作为参数传递，用于初始化类的实例属性；类中定义了一个方法 calculate_change_rate，该方法用于计算本期值相对于基期值的变化率（变动率）。您需要在该类中构造其他方法来计算您所选择的各项指标，关于各项指标的计算公式已在文件 “data/指标计算公式.txt” 中给出，以下是类方法编写的注意事项：
<li>1. 在进行除法运算的过程中, 请注意分母为0的情况, 请将分母为 0 的情况替换为 np.nan, 以避免出现 inf</li>
<li>2. 每种方法只需要传入 self 一个参数，它代表的是类的实例</li>
<li>3. 每种方法需有两个返回值，第一个返回值是所计算指标的名称（str），第二个返回值是各企业的指标值（pandas.Series）</li>
</div>

In [4]:
import numpy as np

class FinancialIndicators:
    def __init__(self, df_2023, df_2022, df_2021):
        self.df_2023 = df_2023
        self.df_2022 = df_2022
        self.df_2021 = df_2021
    
    def calculate_change_rate(self, current, previous):
        '''
        计算本期值相对于基期值的变化率(变动率)

        Args:
            current (pandas.Series): 各企业本期某数据
            previous (pandas.Series): 各企业基期某数据
        
        Returns:
            各企业某指标的变化率 (pandas.Series)
        '''

        ## 若基期数据为0, 则将其替换为nan, 以避免出现inf
        previous = previous.replace(0, np.nan)

        return (current - previous) / previous * 100
    
    '''
    请按照如下示例编写代码计算各项税务风险指标, 示例中的指标为"营业收入变动率", 若您未选择该指标, 可将示例代码注释(选中代码按 Ctrl + / 进行注释)
    注意: 
        1. 在进行除法运算的过程中, 请注意分母为0的情况, 请将分母为0的情况替换为np.nan, 以避免出现inf
        2. “单位产成品原材料耗用率”、“固定资产综合折旧率”、“进项税额变动率与销项税额变动率配比”和“投入产出评估分析指标”四个指标的计算数据可能无法从年报中提取完全, 可以不编写函数进行计算
        3. “印花税税负变动系数”、“印花税同步增长系数”、“资源税税负变动系数”和“资源税同步增长系数”四个指标的计算数据无法从上市公司年报获取, 暂不需要编写函数计算
        4. 每个指标计算函数的返回值为一个元组, 第一个元素为指标名称, 第二个元素为各企业该指标的值, 请按照示例中的格式返回数据
    '''

    ## 1. 营业收入变动率
    '''
    营业收入变动率 = (本期营业收入 - 基期营业收入) / 基期营业收入 * 100%
    '''
    def business_revenue_change_rate(self):
        return '营业收入变动率', self.calculate_change_rate(self.df_2023['营业收入'], self.df_2022['营业收入'])
    
    ##################              请在下方继续添加税务风险指标的计算函数              ##################
    
    ## 2. 单位产成品原材料耗用率 **
    '''
    单位产成品原材料耗用率 = 本期投入原材料 / 本期产成品成本 * 100%
    '''
    # def material_consumption_rate(self):
    #     pass
    
    ## 3. 营业成本变动率
    '''
    营业成本变动率 = (本期营业成本 - 基期营业成本) / 基期营业成本 * 100%
    '''
    def business_cost_change_rate(self):
        return '营业成本变动率', self.calculate_change_rate(self.df_2023['营业成本'], self.df_2022['营业成本'])
    
    ## 4. 营业费用变动率
    '''
    营业费用变动率 = (本期营业费用 - 基期营业费用) / 基期营业费用 * 100%
    营业费用 = 销售费用 + 财务费用 + 管理费用 + 研发费用
    '''
    def business_expense_change_rate(self):
        current_main_business_expense = self.df_2023['销售费用'] + self.df_2023['管理费用'] + self.df_2023['研发费用'] + self.df_2023['财务费用']
        previous_main_business_expense = self.df_2022['销售费用'] + self.df_2022['管理费用'] + self.df_2022['研发费用'] + self.df_2022['财务费用']
        return '营业费用变动率', self.calculate_change_rate(current_main_business_expense, previous_main_business_expense.replace(0, np.nan))
    
    ## 5. 销售费用变动率
    '''
    销售费用变动率 = (本期销售费用 - 基期销售费用) / 基期销售费用 * 100%
    '''
    def operating_expense_change_rate(self): 
        return '销售费用变动率', self.calculate_change_rate(self.df_2023['销售费用'], self.df_2022['销售费用'])
    
    ## 6. 管理费用变动率
    '''
    管理费用变动率 = (本期管理费用 - 基期管理费用) / 基期管理费用 * 100%
    '''
    def management_expense_change_rate(self):
        return '管理费用变动率', self.calculate_change_rate(self.df_2023['管理费用'], self.df_2022['管理费用'])
    
    ## 7. 财务费用变动率
    '''
    财务费用变动率 = (本期财务费用 - 基期财务费用) / 基期财务费用 * 100%
    '''
    def financial_expense_change_rate(self):
        return '财务费用变动率', self.calculate_change_rate(self.df_2023['财务费用'], self.df_2022['财务费用'])
    
    ## 8. 研发费用变动率 
    '''
    研发费用变动率 = (本期研发费用 - 基期研发费用) / 基期研发费用 * 100%
    '''
    def R_and_D_expense_change_rate(self):
        return '研发费用变动率', self.calculate_change_rate(self.df_2023['研发费用'], self.df_2022['研发费用'])
    
    ## 9. 成本费用利润率
    '''
    成本费用利润率 = 利润总额 / 成本费用总额 * 100%
    '''
    def cost_profit_rate(self):
        current_profit = self.df_2023['利润总额']
        current_cost = self.df_2023['营业成本'] + self.df_2023['销售费用'] + self.df_2023['管理费用'] + self.df_2023['研发费用'] + self.df_2023['财务费用']
        return '成本费用利润率', current_profit / current_cost.replace(0, np.nan) * 100
    
    ## 10. 营业利润变动率
    '''
    营业利润变动率 = (本期营业利润 - 基期营业利润) / 基期营业利润 * 100%
    '''
    def business_profit_change_rate(self):
        return '营业利润变动率', self.calculate_change_rate(self.df_2023['营业利润'], self.df_2022['营业利润'])
    
    ## 11. 营业外收入变动率
    '''
    营业外收入变动率 = (本期营业外收入 - 基期营业外收入) / 基期营业外收入 * 100%
    '''
    def non_operating_income_change_rate(self):
        return '营业外收入变动率', self.calculate_change_rate(self.df_2023['加：营业外收入'], self.df_2022['加：营业外收入'])
    
    ## 12. 营业外支出变动率
    '''
    营业外支出变动率 = (本期营业外支出 - 基期营业外支出) / 基期营业外支出 * 100%
    '''
    def non_operating_expense_change_rate(self):
        return '营业外支出变动率', self.calculate_change_rate(self.df_2023['减：营业外支出'], self.df_2022['减：营业外支出'])
    
    ## 13. 净资产收益率
    '''
    净资产收益率 = 净利润 / 平均净资产 * 100%
    '''
    def return_on_equity(self):
        current_net_profit = self.df_2023['净利润']
        current_average_net_assets = (self.df_2023['所有者权益合计'] + self.df_2022['所有者权益合计']) / 2
        return '净资产收益率', current_net_profit / current_average_net_assets.replace(0, np.nan) * 100
    
    ## 14. 总资产周转率
    '''
    总资产周转率 = 营业收入净额 / 平均资产总额 * 100%
    '''
    def total_asset_turnover(self):
        current_net_operating_income = self.df_2023['营业收入']
        current_total_net_assets = (self.df_2023['资产总计'] + self.df_2022['资产总计']) / 2
        return '总资产周转率', current_net_operating_income / current_total_net_assets.replace(0, np.nan) * 100
    
    ## 15. 存货周转率
    '''
    存货周转率 = 营业成本 / ((期初存货成本 + 期末存货成本) / 2) * 100%
    '''
    def inventory_turnover_rate(self):
        return '存货周转率', self.df_2023['营业成本'] / ((self.df_2023['存货净额'] + self.df_2022['存货净额']) / 2).replace(0, np.nan) * 100
    
    ## 16. 应收账款变动率
    '''
    应收账款变动率 = (期末应收账款 - 期初应收账款) / 期初应收账款 * 100%
    '''
    def receivables_change_rate(self):
        return '应收账款变动率', self.calculate_change_rate(self.df_2023['应收账款净额'], self.df_2022['应收账款净额'])
    
    ## 17. 应付账款变动率
    '''
    应付账款变动率 = (期末应付账款 - 期初应付账款) / 期初应付账款 * 100%
    '''
    def payables_change_rate(self):
        return '应付账款变动率', self.calculate_change_rate(self.df_2023['应付账款'], self.df_2022['应付账款'])
    
    ## 18. 其他应收款变动率
    '''
    其他应收款变动率 = (期末其他应收款 - 期初其他应收款) / 期初其他应收款 * 100%
    '''
    def other_receivables_change_rate(self):
        return '其他应收款变动率', self.calculate_change_rate(self.df_2023['其他应收款净额'], self.df_2022['其他应收款净额'])
    
    ## 19. 其他应付款变动率
    '''
    其他应付款变动率 = (期末其他应付款 - 期初其他应付款) / 期初其他应付款 * 100%
    '''
    def other_payables_change_rate(self):
        return '其他应付款变动率', self.calculate_change_rate(self.df_2023['其他应付款'], self.df_2022['其他应付款'])
    
    ## 20. 预收款变动率 **(数据缺失量过大)
    '''
    预收款变动率 = (期末预收款 - 期初预收款) / 期初预收款 * 100%
    '''
    def advance_receipts_change_rate(self):
        return '预收款变动率', self.calculate_change_rate(self.df_2023['预收款项'], self.df_2022['预收款项'])
    
    ## 21. 固定资产综合折旧率 **
    '''
    固定资产综合折旧率 = 基期固定资产折旧总额 / 基期固定资产原值总额 * 100%
    '''
    # def comprehensive_depreciation_rate(self):
    #     pass
    
    ## 22. 资产负债率
    '''
    资产负债率 = 负债总额 / 资产总额 * 100%
    '''
    def asset_liability_rate(self):
        return '资产负债率', self.df_2023['负债合计'] / self.df_2023['资产总计'].replace(0, np.nan) * 100
    
    ## 23. 营业收入变动率与营业利润变动率配比
    '''
    营业收入变动率与营业利润变动率配比 = 营业收入变动率 / 营业利润变动率
    '''
    def business_revenue_profit_ratio_ratio(self):
        current_revenue_change_rate = self.business_revenue_change_rate()[1]
        current_profit_change_rate = self.business_profit_change_rate()[1]
        return '营业收入变动率与营业利润变动率配比', current_revenue_change_rate / current_profit_change_rate
    
    ## 24. 营业收入变动率与营业成本变动率配比
    '''
    营业收入变动率与营业成本变动率配比 = 营业收入变动率 / 营业成本变动率
    '''
    def business_revenue_cost_ratio_ratio(self):
        current_revenue_change_rate = self.business_revenue_change_rate()[1]
        current_cost_change_rate = self.business_cost_change_rate()[1]
        return '营业收入变动率与营业成本变动率配比', current_revenue_change_rate / current_cost_change_rate
    
    ## 25. 营业收入变动率与营业费用变动率配比
    '''
    营业收入变动率与营业费用变动率配比 = 营业收入变动率 / 营业费用变动率
    '''
    def business_revenue_expense_ratio_ratio(self):
        current_revenue_change_rate = self.business_revenue_change_rate()[1]
        current_expense_change_rate = self.business_expense_change_rate()[1]
        return '营业收入变动率与营业费用变动率配比', current_revenue_change_rate / current_expense_change_rate
    
    ## 26. 营业成本变动率与营业利润变动率配比
    '''
    营业成本变动率与营业利润变动率配比 = 营业成本变动率 / 营业利润变动率
    '''
    def business_cost_profit_ratio_ratio(self):
        current_cost_change_rate = self.business_cost_change_rate()[1]
        current_profit_change_rate = self.business_profit_change_rate()[1]
        return '营业成本变动率与营业利润变动率配比', current_cost_change_rate / current_profit_change_rate
    
    ## 27. 资产利润率配比
    '''
    资产利润率配比 = 本期资产利润率 - 上年同期资产利润率
    资产利润率 = (利润总额 / 资产平均占有额) * 100%
    资产平均占有额 = (资产总额期初余额 + 资产总额期末余额) / 2
    '''
    def asset_profit_ratio_ratio(self):
        current_asset_profit_ratio = self.df_2023['利润总额'] / ((self.df_2023['资产总计'] + self.df_2022['资产总计']) / 2).replace(0, np.nan) * 100
        previous_asset_profit_ratio = self.df_2022['利润总额'] / ((self.df_2022['资产总计'] + self.df_2021['资产总计']) / 2).replace(0, np.nan) * 100
        return '资产利润率配比', current_asset_profit_ratio - previous_asset_profit_ratio

    ## 27. 总资产周转率配比
    '''
    总资产周转率配比 = 本期总资产周转率 - 上年同期总资产周转率
    资产周转率 = 本期销售收入净额 / 本期资产总额平均余额 * 100%
    资产总额平均余额 = (资产总额期初余额 + 资产总额期末余额) / 2
    '''
    def total_asset_turnover_ratio_ratio(self):
        current_total_asset_turnover = self.df_2023['营业收入'] / ((self.df_2023['资产总计'] + self.df_2022['资产总计']) / 2).replace(0, np.nan) * 100
        previous_total_asset_turnover = self.df_2022['营业收入'] / ((self.df_2022['资产总计'] + self.df_2021['资产总计']) / 2).replace(0, np.nan) * 100
        return '总资产周转率配比', current_total_asset_turnover - previous_total_asset_turnover
    
    ## 27. 销售利润率配比
    '''
    销售利润率配比 = 本期销售利润率 - 上年同期销售利润率
    销售利润率 = 利润总额 / 营业收入 * 100%
    '''
    def sales_profit_ratio_ratio(self):
        current_sales_profit_ratio = self.df_2023['利润总额'] / self.df_2023['营业收入'].replace(0, np.nan) * 100
        previous_sales_profit_ratio = self.df_2022['利润总额'] / self.df_2022['营业收入'].replace(0, np.nan) * 100
        return '销售利润率配比', current_sales_profit_ratio - previous_sales_profit_ratio
    
    ## 28. 存货变动率配比
    '''
    存货变动率配比 = 本期存货变动率 - 上年同期存货变动率
    存货变动率 = (本期期末库存 - 基期期末库存) / 基期期末库存 * 100%
    '''
    def inventory_change_ratio_ratio(self):
        current_inventory_change_ratio = (self.df_2023['存货净额'] - self.df_2022['存货净额']) / self.df_2022['存货净额'].replace(0, np.nan) * 100
        previous_inventory_change_ratio = (self.df_2022['存货净额'] - self.df_2021['存货净额']) / self.df_2021['存货净额'].replace(0, np.nan) * 100
        return '存货变动率配比', current_inventory_change_ratio - previous_inventory_change_ratio

    ## 29. 增值税税收负担率
    '''
    增值税税收负担率 = (本期应纳税额 / 本期应税营业收入) * 100%
    应纳税额 = 税金及附加 / 12%
    '''
    def value_added_tax_burden_rate(self):
        current_taxable_amount = self.df_2023['税金及附加'] / 0.12
        current_taxable_income = self.df_2023['营业收入']
        return '增值税税收负担率', current_taxable_amount / current_taxable_income.replace(0, np.nan) * 100
    
    ## 30. 应纳税额与工业增加值弹性系数
    '''
    应纳税额与工业增加值弹性系数 = 应纳税额增长率 / 工业增加值增长率
    '''
    def tax_industrial_value_elasticity_coefficient(self):
        current_taxable_amount_change_rate = self.calculate_change_rate(self.df_2023['税金及附加'] / 0.12, self.df_2022['税金及附加'] / 0.12)
        current_industrial_value_change_rate = self.calculate_change_rate(self.df_2023['利润总额'], self.df_2022['利润总额'])
        return '应纳税额与工业增加值弹性系数', current_taxable_amount_change_rate / current_industrial_value_change_rate
    
    ## 31. 工业增加值税负差异率
    '''
    工业增加值税负差异率 = (本企业工业增加值税负 / 同行业工业增加值税负) * 100%
    工业增加值税负 = (本期应纳税额 / 本期利润总额) * 100%
    '''
    def industrial_value_added_tax_burden_difference_rate(self):
        current_industrial_value_added_tax_burden = self.df_2023['税金及附加'] / 0.12 / self.df_2023['利润总额'].replace(0, np.nan) * 100
        return '工业增加值税负差异率', current_industrial_value_added_tax_burden / current_industrial_value_added_tax_burden.mean()

    ## 32. 进项税额变动率与销项税额变动率配比 **
    '''
    进项税额变动率与销项税额变动率配比 = 进项税额变动率 / 销项税额变动率
    '''
    # def input_output_tax_change_rate_ratio(self):
    #     pass

    ## 33. 投入产出评估分析指标 **
    '''
    投入产出评估分析指标 = 当期原材料(燃料、动力等)投入量 / 单位产品原材料(燃料、动力等)使用量
    '''
    # def input_output_evaluation_analysis_index(self):
    #     pass

    ## 34. 所得税负担率
    '''
    所得税负担率 = 应纳所得税额 / 利润总额 * 100%
    '''
    def income_tax_burden_rate(self):
        return '所得税负担率', self.df_2023['减：所得税费用'] / self.df_2023['利润总额'].replace(0, np.nan) * 100
    
    ## 35. 营业利润税收负担率
    '''
    营业利润税收负担率 = (本期应纳所得税额 / 本期营业利润) * 100%
    '''
    def business_profit_tax_burden_rate(self):
        return '营业利润税收负担率', self.df_2023['减：所得税费用'] / self.df_2023['营业利润'].replace(0, np.nan) * 100
    
    ## 36. 应纳税所得额变动率 **
    '''
    应纳税所得额变动率 = (评估期累计应纳税所得额 - 基期累计应纳税所得额) / 基期累计应纳税所得额 * 100%
    财报中没有＂应纳税所得额＂，可用＂利润总额＂替代
    '''
    def taxable_income_change_rate(self):
        return '应纳税所得额变动率', self.calculate_change_rate(self.df_2023['利润总额'], self.df_2022['利润总额'])

    ## 37. 所得税贡献率
    '''
    所得税贡献率 = 应纳所得税额 / 营业收入 * 100%
    '''
    def income_tax_contribution_rate(self):
        return '所得税贡献率', self.df_2023['减：所得税费用'] / self.df_2023['营业收入'].replace(0, np.nan) * 100
    
    ## 38. 所得税贡献变动率
    '''
    所得税贡献变动率 = (评估期所得税贡献率 - 基期所得税贡献率) / 基期所得税贡献率 * 100%
    '''
    def income_tax_contribution_change_rate(self):
        current_income_tax_contribution_rate = self.df_2023['减：所得税费用'] / self.df_2023['营业收入'].replace(0, np.nan) * 100
        previous_income_tax_contribution_rate = self.df_2022['减：所得税费用'] / self.df_2022['营业收入'].replace(0, np.nan) * 100
        return '所得税贡献变动率', self.calculate_change_rate(current_income_tax_contribution_rate, previous_income_tax_contribution_rate)

    ## 39. 所得税负担变动率
    '''
    所得税负担变动率 = (评估期所得税负担率 - 基期所得税负担率) / 基期所得税负担率 * 100%
    '''
    def income_tax_burden_change_rate(self):
        current_income_tax_burden_rate = self.df_2023['减：所得税费用'] / self.df_2023['利润总额'].replace(0, np.nan) * 100
        previous_income_tax_burden_rate = self.df_2022['减：所得税费用'] / self.df_2022['利润总额'].replace(0, np.nan) * 100
        return '所得税负担变动率', self.calculate_change_rate(current_income_tax_burden_rate, previous_income_tax_burden_rate)
    
    ## 40. 印花税税负变动系数 ** 
    '''
    印花税税负变动系数 = 本期印花税负担率 / 上年同期印花税负担率
    印花税负担率 = (应纳税额 / 计税收入) * 100%
    '''
    # def stamp_tax_burden_change_coefficient(self):
    #     pass

    ## 41. 印花税同步增长系数
    '''
    印花税同步增长系数 = 应纳税额增长率 / 营业收入增长率
    '''
    # def stamp_tax_synchronous_growth_coefficient(self):
    #     current_taxable_amount_change_rate = self.calculate_change_rate(self.df_2023['税金及附加'] / 0.12, self.df_2022['税金及附加'] / 0.12)
    #     current_revenue_change_rate = self.calculate_change_rate(self.df_2023['营业收入'], self.df_2022['营业收入'])
    #     return '印花税同步增长系数', current_taxable_amount_change_rate / current_revenue_change_rate
    
    ## 42. 资源税税负变动系数 **
    '''
    资源税税负变动系数 = 本期资源税税收负担率 / 上年同期资源税税收负担率
    '''
    # def resource_tax_burden_change_coefficient(self):
    #     pass

    ## 43. 资源税同步增长系数 **
    '''
    资源税同步增长系数 = 应纳税额增长率 / 营业收入(产品销售收入)增长率
    '''
    # def resource_tax_synchronous_growth_coefficient(self):
    #     pass

    ##################              请在上方继续添加税务风险指标的计算函数              ##################

## 实例化类
indicators = FinancialIndicators(df_2023, df_2022, df_2021)

## 获取类中的所有方法名
def get_class_methods(cls):
    attr_names = [attr_name for attr_name in dir(cls) if callable(getattr(cls, attr_name)) and not attr_name.startswith("__") and attr_name != 'calculate_change_rate']
    return attr_names

attr_names = get_class_methods(FinancialIndicators)

## 创建一个字典, 用于存储各项税务风险指标的计算结果
dicts = {
    '证券代码': df_2023['证券代码'],
    '证券简称': df_2023['证券简称']
}

## 计算各项税务风险指标
for attr_name in attr_names:
    v_name, values = getattr(indicators, attr_name)()
    dicts[v_name] = values

## 将计算结果转换为DataFrame
df = pd.DataFrame(dicts)

# df.head()

## indicators.csv: 存放企业税务风险指标数据
df.to_csv('output/indicators.csv', encoding = 'utf-8-sig', index = False)

<a id = "ref13"></a>
### 步骤 2：计算行业税务风险指标预警值

<a id = "ref13"></a>
### 行业预警值计算说明
<div class="alert alert-block alert-info" style="margin-top: 20px">
行业税务风险指标预警值的计算分为以下四个步骤：
<li>1. 计算出各企业所选指标的平均值。以指标“营业收入变动率”为例，则行业的平均“营业收入变动率”为：</li>

$$ \overline{x}=\frac{1}{n} \sum_{i=1}^{n}x_i $$

其中 $x_i$ 表示第 $i$ 个企业的“营业收入变动率”，$n$ 为所统计企业总数（如果某企业无法计算出该指标则不参与计数）
<li>2. 计算出各企业所选指标的标准差：</li>

$$ \sigma_x=\sqrt{\frac{\sum_{i=1}^{n}(x_i-\overline{x})^2 }{n} } $$

<li>3. 计算出所选指标的离散系数：</li>

$$ cv_x=\frac{\sigma_x}{\overline{x}} $$

<li>4. 计算出所选指标的行业预警值</li>
<ul class = "toc-item">
        4.1. 非配比指标行业预警值计算：

$$ warning_x=\begin{equation}\left\{\begin{array}{lr}(\overline{x}-\sigma_x,\overline{x}+\sigma_x), &  cv_x\le 0.6 \\(\overline{x}-0.6\times\sigma_x,\overline{x}+0.6\times\sigma_x), &  cv_x>0.6 \end{array}\right.\end{equation} $$
</ul>
<ul class = "toc-item">
        4.2. 配比指标行业预警值计算：

$$ warning_x=(0.7,1.3) $$
注：配比指标评分标准根据《关于印发<纳税评估管理办法（试行）>的通知》，以系数1为行业均值，预警值下限为0.7，预警值上限为1.3。这里的“营业收入变动率”为非配比指标，故应该按照4.1中的公式计算其行业预警值。
</ul>
</div>

<div class="alert alert-block alert-info" style="margin-top: 20px">
&emsp;&emsp;关于具体代码的编写，下方预定义了一个名为 Warning_level 的函数，该函数是用来计算某项税务风险指标的预警值，所以需传入的其中一个参数是所选行业部分企业某税务风险指标，同时还需要传入一个参数来记录该指标的类型（配比或非配比）。但在实际计算过程中可能会用到其他参数，您可根据自身需求进行添加。函数返回该项税务风险指标的预警值以及预警值计算过程中使用到的部分中间变量。关于输入和输出内容的格式已在函数中定义。请完成该函数，下面是一些注意事项：
<li>1. 您需要在函数中完成对所选择指标是配比指标还是非配比指标的判断，关于您所选指标的分类已存储在 ratio_ratio_group_infos 中，您可运行下方代码查看该变量</li>
<li>2. 对于某项指标，并不是所有企业的该项指标值都位于正常区间，您需要采取某种方式去除异常值</li>
</div>

In [5]:
import json

## 载入对所选指标的分组信息（配比或非配比）
with open('data/配比指标划分.json', 'r') as file:
    ratio_ratio_group_infos = json.load(file)

'''
ratio_ratio_group_infos = {
    'non_ratio_ratio_indicator': ['indicator1', 'indicator2', 'indicator3', ...],  ## 非配比指标
    'ratio_ratio_indicator': ['indicator4', 'indicator5', 'indicator6', ...]  ## 配比指标
}
'''
# print(ratio_ratio_group_infos)

def Warning_level(indicator, ratio_ratio_group_infos, **kwargs):
    '''
    计算某项税务风险指标的预警值

    Args:
        indicator (pandas.Series): 所选行业部分企业某税务风险指标
        ratio_ratio_group_infos (dict): 所选指标的分组信息（配比或非配比）
    
    Returns:
        tuple: 包含 ('行业均值', '行业标准差', '行业离散系数', '预警值下限', '预警值上限')

    注意: 配比指标请返回 (1, np.nan, np.nan, 0.7, 1.3)
    '''

    if indicator.name in ratio_ratio_group_infos['non_ratio_ratio_indicator']:
        Q1, Q3 = indicator.quantile(0.25), indicator.quantile(0.75)
        IQR = Q3 - Q1

        lower_bound = Q1 - kwargs['multiplier'] * IQR
        upper_bound = Q3 + kwargs['multiplier'] * IQR

        selected = (indicator > lower_bound) & (indicator < upper_bound)
        # print(indicator[~selected])
        # print('"' + indicator.name + '"指标异常企业总数为: ', len(indicator[~selected]))

        indicator = indicator[selected]
        
        ave = indicator.mean()
        sd = indicator.std()
        # print('均值: ', ave)
        # print('标准差: ', sd)

        cv = sd / ave
        # print('离散系数: ', cv)

        if cv <= 0.6:
            return (ave, sd, cv, ave - sd, ave + sd)
        else:
            return (ave, sd, cv, ave - 0.6 * sd, ave + 0.6 * sd)
    
    elif indicator.name in ratio_ratio_group_infos['ratio_ratio_indicator']:
        return (1, np.nan, np.nan, 0.7, 1.3)

    else:
        raise ValueError(f"指标名称 '{indicator.name}' 错误，不在配比或非配比指标列表中。")

result_warning = df.iloc[:,2:].apply(lambda x: Warning_level(x, ratio_ratio_group_infos, multiplier = 1.5))
result_warning.index = ['行业均值', '行业标准差', '行业离散系数', '预警值下限', '预警值上限']
# result_warning

## result_warning.csv: 存放行业税务风险指标预警值
result_warning.to_csv('output/result_warning.csv', encoding = 'utf-8-sig')